# First look at the data


In [18]:
import pandas as pd
import numpy as np

# We first take a look at the data file, and understand that it is a CSV file with 28 KB on your disk

# read the file
covid_data = pd.read_csv("data/covidtotals.csv", parse_dates=["lastdate"]) # note: Pandas can automatically infer standard date formats, such as ISO 8601 (YYYY-MM-DD), if you specify which columns contain dates.

In [19]:
# Inspect the shape (numbers of rows and columns) of the dataframe.
# pd.shape returns a tuple representing the dimensionality of the DataFrame.
# it has 231 rows and 17 columns
covid_data.shape

(231, 17)

In [20]:
# Inspect the columns

print(f"This file include the following columns: {covid_data.columns}")

This file include the following columns: Index(['iso_code', 'lastdate', 'location', 'total_cases', 'total_deaths',
       'total_cases_pm', 'total_deaths_pm', 'population', 'pop_density',
       'median_age', 'gdp_per_capita', 'hosp_beds', 'vac_per_hund',
       'aged_65_older', 'life_expectancy', 'hum_dev_ind', 'region'],
      dtype='object')


In [21]:
# Inspect some samples of the data, using head( ), tail ( ), or sample( )
print(f"The head of the dataset:\n")
print(covid_data.head())

The head of the dataset:

  iso_code   lastdate        location  total_cases  total_deaths  \
0      AFG 2024-02-04     Afghanistan     231539.0        7982.0   
1      ALB 2024-01-28         Albania     334863.0        3605.0   
2      DZA 2023-12-03         Algeria     272010.0        6881.0   
3      ASM 2023-09-17  American Samoa       8359.0          34.0   
4      AND 2023-05-07         Andorra      48015.0         159.0   

   total_cases_pm  total_deaths_pm  population  pop_density  median_age  \
0        5629.611          194.073    41128772       54.422        18.6   
1      117813.348         1268.331     2842318      104.871        38.0   
2        6057.694          153.241    44903228       17.348        29.1   
3      188712.044          767.581       44295      278.205         NaN   
4      601367.684         1991.408       79843      163.755         NaN   

   gdp_per_capita  hosp_beds  vac_per_hund  aged_65_older  life_expectancy  \
0        1803.987       0.50        

In [15]:
print(f"The tail of the dataset:\n")
print(covid_data.tail(n=3)) # You can specify how many rows you wanted to inspect

The tail of the dataset:

    iso_code    lastdate  location  total_cases  total_deaths  total_cases_pm  \
228      YEM  2022-11-06     Yemen      11945.0        2159.0         354.487   
229      ZMB  2023-12-03    Zambia     349304.0        4069.0       17449.783   
230      ZWE  2024-01-28  Zimbabwe     266265.0        5737.0       16314.719   

     total_deaths_pm  population  pop_density  median_age  gdp_per_capita  \
228           64.072    33696612       53.508        20.3        1479.147   
229          203.270    20017670       22.995        17.7        3689.251   
230          351.520    16320539       42.729        19.6        1899.775   

     hosp_beds  vac_per_hund  aged_65_older  life_expectancy  hum_dev_ind  \
228        0.7           NaN          2.922            66.12        0.470   
229        2.0           NaN          2.480            63.89        0.584   
230        1.7           NaN          2.822            61.49        0.571   

              region  
228     

In [16]:
print(f"A random sample of the dataset:\n")
print(covid_data.sample())

A random sample of the dataset:

   iso_code    lastdate location  total_cases  total_deaths  total_cases_pm  \
21      BEN  2023-09-24    Benin      28036.0         163.0        2099.624   

    total_deaths_pm  population  pop_density  median_age  gdp_per_capita  \
21           12.207    13352864        99.11        18.8        2064.236   

    hosp_beds  vac_per_hund  aged_65_older  life_expectancy  hum_dev_ind  \
21        0.5           NaN          3.244            61.77        0.545   

         region  
21  West Africa  


In [17]:
print(f"A random sample of the datasets with 7 rows, with random seed 42")
print(covid_data.sample(n=7, random_state=42))

A random sample of the datasets with 7 rows, with random seed 42
    iso_code    lastdate                         location  total_cases  \
218      GBR  2024-01-28                   United Kingdom   24892903.0   
66       ETH  2023-12-03                         Ethiopia     501117.0   
9        ARM  2023-12-17                          Armenia     451426.0   
170      RUS  2024-01-28                           Russia   23774451.0   
15       BHR  2022-12-04                          Bahrain     696614.0   
200      SWE  2024-01-28                           Sweden    2749793.0   
25       BES  2023-07-09  Bonaire Sint Eustatius and Saba      11922.0   

     total_deaths  total_cases_pm  total_deaths_pm  population  pop_density  \
218      232112.0      368734.933         3438.241    67508936      272.898   
66         7574.0        4061.576           61.388   123379928      104.957   
9          8775.0      162355.888         3155.939     2780472      102.931   
170      401884.0      164

In [ ]:
%%sql


In [24]:
covid_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 231 entries, 0 to 230
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   iso_code         231 non-null    object        
 1   lastdate         231 non-null    datetime64[ns]
 2   location         231 non-null    object        
 3   total_cases      231 non-null    float64       
 4   total_deaths     231 non-null    float64       
 5   total_cases_pm   231 non-null    float64       
 6   total_deaths_pm  231 non-null    float64       
 7   population       231 non-null    int64         
 8   pop_density      209 non-null    float64       
 9   median_age       194 non-null    float64       
 10  gdp_per_capita   191 non-null    float64       
 11  hosp_beds        170 non-null    float64       
 12  vac_per_hund     13 non-null     float64       
 13  aged_65_older    188 non-null    float64       
 14  life_expectancy  227 non-null    float64  

In [26]:
covid_data.dtypes

iso_code                   object
lastdate           datetime64[ns]
location                   object
total_cases               float64
total_deaths              float64
total_cases_pm            float64
total_deaths_pm           float64
population                  int64
pop_density               float64
median_age                float64
gdp_per_capita            float64
hosp_beds                 float64
vac_per_hund              float64
aged_65_older             float64
life_expectancy           float64
hum_dev_ind               float64
region                     object
dtype: object

## Summary statistics (numerical)


In [33]:
covid_data.describe() # generate a descriptive statistics for numerical columns

,lastdate,total_cases,total_deaths,total_cases_pm,total_deaths_pm,population,pop_density,median_age,gdp_per_capita,hosp_beds,vac_per_hund,aged_65_older,life_expectancy,hum_dev_ind
count,231,2.310000e+02,2.310000e+02,231.000000,231.000000,2.310000e+02,209.000000,194.000000,191.000000,170.000000,13.000000,188.000000,227.00000,187.000000
mean,2023-09-11 22:32:43.636363520,3.351599e+06,3.021420e+04,206177.794623,1261.778390,3.424609e+07,323.603359,30.341753,18614.958990,3.014576,196.476923,8.675835,73.64652,0.721251
min,2020-10-18 00:00:00,4.000000e+00,0.000000e+00,354.487000,0.000000,4.700000e+01,0.137000,15.100000,661.240000,0.100000,38.470000,1.144000,53.28000,0.394000
25%,2023-06-07 12:00:00,2.567150e+04,1.775000e+02,21821.863000,141.177000,3.702075e+05,37.728000,22.050000,3821.198000,1.300000,155.720000,3.496750,69.54500,0.601500
50%,2023-12-03 00:00:00,1.914960e+05,1.937000e+03,133946.251000,827.046000,5.434324e+06,88.125000,29.600000,12236.706000,2.398500,214.070000,6.258500,75.05000,0.740000
75%,2024-01-28 00:00:00,1.294286e+06,1.415000e+04,345689.831000,1997.513000,2.197870e+07,222.873000,38.700000,27012.304500,3.965250,227.810000,13.990500,79.28500,0.828500
max,2024-02-04 00:00:00,1.034368e+08,1.127152e+06,763475.441000,6507.656000,1.425887e+09,19347.500000,48.200000,116935.600000,13.800000,406.390000,27.049000,86.75000,0.957000
std,NaN,1.148321e+07,1.047789e+05,203858.096252,1314.981529,1.376530e+08,1468.427340,9.117782,19470.167828,2.434476,96.305392,6.150235,7.42389,0.149669


In [34]:
covid_data.describe(percentiles=[0.05, 0.95])

,lastdate,total_cases,total_deaths,total_cases_pm,total_deaths_pm,population,pop_density,median_age,gdp_per_capita,hosp_beds,vac_per_hund,aged_65_older,life_expectancy,hum_dev_ind
count,231,2.310000e+02,2.310000e+02,231.000000,231.000000,2.310000e+02,209.000000,194.000000,191.000000,170.000000,13.000000,188.000000,227.00000,187.000000
mean,2023-09-11 22:32:43.636363520,3.351599e+06,3.021420e+04,206177.794623,1261.778390,3.424609e+07,323.603359,30.341753,18614.958990,3.014576,196.476923,8.675835,73.64652,0.721251
min,2020-10-18 00:00:00,4.000000e+00,0.000000e+00,354.487000,0.000000,4.700000e+01,0.137000,15.100000,661.240000,0.100000,38.470000,1.144000,53.28000,0.394000
5%,2022-10-23 00:00:00,5.239000e+03,8.000000e+00,1638.859000,13.193000,1.428400e+04,5.245800,17.665000,1423.126500,0.500000,57.196000,2.482100,59.50500,0.462300
50%,2023-12-03 00:00:00,1.914960e+05,1.937000e+03,133946.251000,827.046000,5.434324e+06,88.125000,29.600000,12236.706000,2.398500,214.070000,6.258500,75.05000,0.740000
95%,2024-02-04 00:00:00,1.549251e+07,1.447630e+05,600293.033500,3749.868000,1.194695e+08,675.255200,44.140000,55543.458000,7.212500,324.538000,19.721900,82.98400,0.938000
max,2024-02-04 00:00:00,1.034368e+08,1.127152e+06,763475.441000,6507.656000,1.425887e+09,19347.500000,48.200000,116935.600000,13.800000,406.390000,27.049000,86.75000,0.957000
std,NaN,1.148321e+07,1.047789e+05,203858.096252,1314.981529,1.376530e+08,1468.427340,9.117782,19470.167828,2.434476,96.305392,6.150235,7.42389,0.149669


## Summary statistics (categorical)